In [1]:
%load_ext autoreload
%autoreload 2

## Information Retrieval Demo

Date: 26 April 2023

In this demo, we will be demo-ing how to perform Information Retrieval using Langchain.

### What is LangChain

LangChain is a framework for developing applications powered by language models. 

_LangChain Principles:_
- Be data-aware: connect a language model to other sources of data
- Be agentic: allow a language model to interact with its environment

Source: [LangChain documentation](https://python.langchain.com/en/latest/index.html)


## Document Retrieval

<img src="https://substackcdn.com/image/fetch/w_1272,c_limit,f_webp,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F4063347e-8920-40c6-86b3-c520084b303c_1272x998.jpeg" alt= “” width="600" height="500">


<!-- ![](https://substackcdn.com/image/fetch/w_1272,c_limit,f_webp,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F4063347e-8920-40c6-86b3-c520084b303c_1272x998.jpeg) -->

Source: [Finetuning Large Language Models by Sebastian Raschka](https://magazine.sebastianraschka.com/p/finetuning-large-language-models)

## LangChain: Models

Generic inferface for models e.g. LLMs, Chat Models, Text Embedding Models. Read more [here](https://python.langchain.com/en/latest/modules/models.html)

```python
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-ada-001")

from langchain import HuggingFaceHub
llm = HuggHuggingFaceHub(repo_id="google/flan-t5-xl")

from langchain.llms import Cohere
llm = Cohere()
```

## Local Models: HuggingFace Pipeline

Note: This is experimental code, always use functional or OO abstraction for your implementation.

```python
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM

model_id = 'flan-t5-large' # any local model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True) # 8bit in A10, A100 etc.

pipe = pipeline(
    task="text2text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)
```

## LangChain: Prompts

Prompt Management, Optimization and Serialization. Read more [here](https://docs.langchain.com/docs/components/prompts)

```python
from langchain import PromptTemplate

template = """Question: {question}

Let's think step by step.

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])
user_input = input("Enter your question: ")
prompt.format(question=user_input)
```

## LangChain: Chain

Sequence of calls (with multiple models). Read more [here](https://python.langchain.com/en/latest/modules/chains.html)  
[] `LLMChain`  
[] `SequentialChain`  
[] Custom `Chain`  

```python
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="Which country is the largest producer of {product}?",
)

from langchain.chains import LLMChain
# Chain a LLM with a prompt template
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("lithium"))
```


```python
from langchain.chains import SimpleSequentialChain

q = "What type of mammal lays the biggest eggs?"

llm = OpenAI(temperature=0.7)
template = """{question}\n\n"""
prompt_template = PromptTemplate(input_variables=["question"], template=template)
question_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """Here is a statement:
{statement}
Make a bullet point list of the assumptions you made when producing the above statement.\n\n"""
prompt_template = PromptTemplate(input_variables=["statement"], template=template)
assumptions_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """Here is a bullet point list of assertions:
{assertions}
For each assertion, determine whether it is true or false. If it is false, explain why.\n\n"""
prompt_template = PromptTemplate(input_variables=["assertions"], template=template)
fact_checker_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """In light of the above facts, how would you answer the question '{}'""".format(q)
template = """{facts}\n""" + template
prompt_template = PromptTemplate(input_variables=["facts"], template=template)
answer_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(chains=[question_chain, assumptions_chain, fact_checker_chain, answer_chain], verbose=True)
```

Credit to jagilley/fact-checker for the example. Check the repo [here](https://github.com/jagilley/fact-checker).


![Importance of SeqChains](https://weaviate.io/assets/images/sequential-chains-fec82f27b64a0d8f5b6123b39569ecf2.gif)

## LangChain: Agents

Agents that can use tools like Google Search or Wikipedia. Read more [here](https://python.langchain.com/en/latest/modules/agents.html)

Note: This is _not_ great for air gapped systems.

```python
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.agents import 

from langchain.llms import OpenAI
llm = OpenAI(temperature=0)

tools = load_tools(["google-search", "wikimedia", "llm-math"], 
                    llm=llm)
agent = initialize_agent(tools=tools,
                        llm=llm,
                        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                        verbose=True)

agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?") #relevant example
```

## LangChain: Memory

Momory components for
1. utilities for managing and manipulating previous chat messages
2. incorporate these utilities into chains

Read more [here](https://python.langchain.com/en/latest/modules/memory.html).

## LangChain: Indexes

Utility functions to combine own private data. Read more [here](https://python.langchain.com/en/latest/modules/indexes.html).

Now we are talking.

### [Document Loaders](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html)

Email, Images, PDF, s3 Directory and Files, Word Documents, Powerpoints etc.

### [Text Splitters](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html)

Character Text Splitter, Recursive Character Text Splitter here.

### [VectorStores](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html)

ElasticSearch, FAISS, Qdrant, Redis, Weaviate

### [Retrievers](https://python.langchain.com/en/latest/modules/indexes/retrievers.html)

Contextual Compression Retriever (!!!), SVM Retriever, TF-IDF Retriever, Time Weighted VectorStore Retriever etc.



## LangChain: Indexes

```python
query = <YOUR_QUERY_HERE>

from langchain.vectorstores import FAISS
db_faiss = FAISS.from_documents(documents=docs, 
                                embedding=embeddings)
docs = db_faiss.similarity_search(query)

from langchain.vectorstores import ElasticVectorSearch
db_elastic = ElasticVectorSearch.from_documents(documents=docs, 
                                                embedding=embeddings, 
                                                elasticsearch_url="http://localhost:9200")
docs = db_elastic.similarity_search(query)
```

In [7]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id,)# load_in_8bit=True)

pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [2]:
print(local_llm('What is the capital of France? '))

paris


In [12]:
print(local_llm("Which country is the biggest producer of wine?"))


switzerland


In [13]:
print(local_llm("Which country is the biggest producer of wine? \n Let's think step by step."))


The United States is the biggest producer of wine. The United States is the largest producer of wine in the world. So, the answer is United States.


In [14]:
print(local_llm("Which country is the biggest producer of wine? How much wine does this country make compared to second biggest producer of wine?\n Let's think step by step."))

The biggest producer of wine is France. France makes a total of 1.2 billion bottles of wine a year. The second biggest producer of wine is Italy. Italy makes 1.2 billion bottles of wine a year. The answer: Italy.


## References

[1] LangChain Documentation